# 패키지 설치하기

In [ ]:
!pip install ratsnlp

In [ ]:
!pip install --upgrade pytorch-lightning

# 환경 설정

모델 hyperparameter와 저장 위치 등 설정 정보를 선언

In [5]:
from ratsnlp.nlpbook.qa import QADeployArguments

In [7]:
args = QADeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_dir="/content/drive/Othercomputers/내 컴퓨터/Chapter 7. Question Answering/checkpoint-qa",
    max_seq_length=128,
    max_query_length=32,
)

downstream_model_checkpoint_fpath: /content/drive/Othercomputers/내 컴퓨터/Chapter 7. Question Answering/checkpoint-qa/epoch=0-val_loss=0.46.ckpt


# 모델 불러오기

파인튜닝을 마친 모델 불러오기

In [8]:
import torch
from transformers import BertConfig, BertForQuestionAnswering

In [10]:
# 체크포인트 로드
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu")
)

# BERT 설정 로드
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
)

model = BertForQuestionAnswering(pretrained_model_config)      # 모델 초기화
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})     # 체크포인트 주입
model.eval()    # 평가모드

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

토크나이저 준비

In [11]:
from transformers import BertTokenizer

In [12]:
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

# 인퍼런스 함수

In [13]:
def inference_fn(question, context):
    if question and context:

        # 질문을 토큰화, 길이가 길면 자르기
        truncated_query = tokenizer.encode(
            question,
            add_special_tokens=False,
            truncation=True,
            max_length=args.max_query_length
        )

        # 앞서 처리한 질문을 지문과 함께 토큰화, 길이가 길면 자르기
        inputs = tokenizer.encode_plus(
            text=truncated_query,
            text_pair=context,
            truncation="only_second",
            padding="max_length",
            max_length=args.max_seq_length,
            return_token_type_ids=True,
        )

        with torch.no_grad():
            outputs = model(**{k: torch.tensor([v]) for k, v in inputs.items()})    # 토큰화 된 데이터를 텐서로 변환 후 모델에 입력
            start_pred = outputs.start_logits.argmax(dim=-1).item()     # 정답의 시작 위치와 관련 된 로짓에서 가장 큰 값을 가진 토큰 찾기
            end_pred = outputs.end_logits.argmax(dim=-1).item()         # 정답의 끝 위치와 관련 된 로짓에서 가장 큰 값을 가진 토큰 찾기
            pred_text = tokenizer.decode(inputs['input_ids'][start_pred:end_pred+1])        # 정답 시작토큰 부터 끝토큰 이어붙이기
    else:
        pred_text = ""
    return {
        'question': question,
        'context': context,
        'answer': pred_text,
    }

# 웹서비스 개시

`ngrok` 은 코랩 로컬에서 실행 중인 웹서비스를 안전하게 외부에서 접근 가능하도록 해주는 도구이다. 실행하려면 [회원가입](https://dashboard.ngrok.com/signup) 후 [로그인](https://dashboard.ngrok.com/login)을 한 뒤 [이곳](https://dashboard.ngrok.com/get-started/your-authtoken)에 접속해 인증 토큰(authtoken)을 입력해야 한다.

In [14]:
!mkdir /root/.ngrok2 && echo "authtoken: {이곳에 확인된 인증 토큰을 입력하세요}" > /root/.ngrok2/ngrok.yml

인퍼런스 함수를 Flask를 통해 웹서비스로 만든다.

In [17]:
from ratsnlp.nlpbook.qa import get_web_service_app

In [18]:
app = get_web_service_app(inference_fn)
app.run()

 * Serving Flask app "ratsnlp.nlpbook.qa.deploy" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://58ed-34-125-228-231.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [15/Apr/2022 06:55:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2022 06:55:27] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [15/Apr/2022 06:55:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2022 06:58:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2022 06:59:29] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2022 06:59:46] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2022 07:00:15] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2022 07:01:28] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2022 07:01:59] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2022 07:02:18] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2022 07:02:47] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2022 07:04:11] "POST /api HTTP/1.1" 200 -
